In [ ]:
pip install findspark

In [ ]:
pip install pyspark

In [1]:
import pyspark
from pyspark.sql import SparkSession
#SparkSession.builder
 #    .master("local")
  #   .appName("Word Count")
   #  .config("spark.some.config.option", "some-value")
   #  .getOrCreate()

spark = SparkSession.builder.getOrCreate()
df = spark.sql("select 'spark' as hello ")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [2]:
spark

In [84]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

sql_sc = SQLContext(spark)
repository='C:/Users/adam_/Desktop/Adam_Work_Spark/trips_spark/Shared_Micromobility_Vehicle_Trips.csv'
Austin_Trips_Df = spark.read.csv(repository,header='True',inferSchema='true') 


In [86]:
Austin_Trips_Df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Device ID: string (nullable = true)
 |-- Vehicle Type: string (nullable = true)
 |-- Trip Duration: integer (nullable = true)
 |-- Trip Distance: integer (nullable = true)
 |-- Start Time: string (nullable = true)
 |-- End Time: string (nullable = true)
 |-- Modified Date: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Day of Week: integer (nullable = true)
 |-- Council District (Start): integer (nullable = true)
 |-- Council District (End): integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Census Tract Start: string (nullable = true)
 |-- Census Tract End: string (nullable = true)



In [30]:
#from pyspark.sql.functions import to_timestamp
#Austin_Trips_Df = Austin_Trips_Df.withColumn('Start_Time',to_timestamp(Austin_Trips_Df['Start Time'], "yyyy_MM_dd HH_mm_ss"))

In [87]:
Austin_Trips_Df.head(1)

[Row(ID='900f1254-8a1e-495e-a55f-1153ebdb3e0e', Device ID='4f7f79d1-88d8-4829-95be-5a548134425b', Vehicle Type='bicycle', Trip Duration=188, Trip Distance=836, Start Time='07/01/2019 12:00:00 AM', End Time='07/01/2019 12:00:00 AM', Modified Date='07/03/2019 05:02:16 AM', Month=7, Hour=0, Day of Week=1, Council District (Start)=3, Council District (End)=3, Year=2019, Census Tract Start='48453002111', Census Tract End='48453002111')]

In [88]:
Austin_Trips_Df=Austin_Trips_Df.withColumn('Trip Distance',(Austin_Trips_Df['Trip Distance'])/1000).withColumn('Trip Duration',(Austin_Trips_Df['Trip Duration'])/3600)

In [89]:
Austin_Trips_Df=Austin_Trips_Df.withColumnRenamed('Trip Distance', 'Trip_Distance(km)').withColumnRenamed('Trip Duration', 'Trip_Duration(H)')

In [90]:
Austin_Trips_Df=Austin_Trips_Df.drop(Austin_Trips_Df['Modified Date'])

In [91]:
Austin_Trips_Df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Device ID: string (nullable = true)
 |-- Vehicle Type: string (nullable = true)
 |-- Trip_Duration(H): double (nullable = true)
 |-- Trip_Distance(km): double (nullable = true)
 |-- Start Time: string (nullable = true)
 |-- End Time: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Day of Week: integer (nullable = true)
 |-- Council District (Start): integer (nullable = true)
 |-- Council District (End): integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Census Tract Start: string (nullable = true)
 |-- Census Tract End: string (nullable = true)



In [92]:
Austin_Trips_Df.count()

2115445

In [93]:
Austin_Trips_Df.describe().show(truncate= True)

+-------+--------------------+--------------------+------------+-------------------+-----------------+--------------------+--------------------+------------------+------------------+------------------+------------------------+----------------------+-------+--------------------+--------------------+
|summary|                  ID|           Device ID|Vehicle Type|   Trip_Duration(H)|Trip_Distance(km)|          Start Time|            End Time|             Month|              Hour|       Day of Week|Council District (Start)|Council District (End)|   Year|  Census Tract Start|    Census Tract End|
+-------+--------------------+--------------------+------------+-------------------+-----------------+--------------------+--------------------+------------------+------------------+------------------+------------------------+----------------------+-------+--------------------+--------------------+
|  count|             2115445|             2115445|     2115445|            2115445|          211544

In [81]:
Austin_Trips_Df.describe(['Trip_Distance(km)']).show(),Austin_Trips_Df.describe(['Trip_Duration(H)']).show()

+-------+-----------------+
|summary|Trip_Distance(km)|
+-------+-----------------+
|  count|          2115445|
|   mean|45.60924962360113|
| stddev|1936.536590271756|
|    min|              0.0|
|    max|      1113117.831|
+-------+-----------------+

+-------+-------------------+
|summary|   Trip_Duration(H)|
+-------+-------------------+
|  count|            2115445|
|   mean|0.16955681625694452|
| stddev| 0.2429257070840986|
|    min|             -0.315|
|    max|            23.5475|
+-------+-------------------+



(None, None)

In [85]:
Austin_Trips_Df.filter(Austin_Trips_Df['Trip Duration']<0).show()

+--------------------+--------------------+------------+-------------+-------------+--------------------+--------------------+--------------------+-----+----+-----------+------------------------+----------------------+----+------------------+----------------+
|                  ID|           Device ID|Vehicle Type|Trip Duration|Trip Distance|          Start Time|            End Time|       Modified Date|Month|Hour|Day of Week|Council District (Start)|Council District (End)|Year|Census Tract Start|Census Tract End|
+--------------------+--------------------+------------+-------------+-------------+--------------------+--------------------+--------------------+-----+----+-----------+------------------------+----------------------+----+------------------+----------------+
|cf754985-0a8c-4e6...|bd9f190d-2509-433...|     scooter|           -9|          643|07/18/2019 06:45:...|07/18/2019 06:45:...|07/19/2019 08:28:...|    7|  18|          4|                       3|                     3|20

In [ ]:
Austin_Trips_Df.select('Vehicle Type').distinct().show()

In [ ]:
Austin_Trips_Df.crosstab('Vehicle Type', 'Month').show()

In [ ]:
Austin_Trips_Df.crosstab('Day of Week', 'Month').show()

In [ ]:
Austin_Trips_Df.crosstab('Hour','Day of Week').show()

In [ ]:
Austin_Trips_Df.crosstab('Device ID','Month').show()

In [ ]:
Austin_Trips_Df.dropna().count()

In [ ]:
Austin_Trips_Df.withColumn('Speed',(Austin_Trips_Df['Trip Distance']/Austin_Trips_Df['Trip Duration'])*3.6).select('Trip Duration','Trip Distance','Speed').show(5)

In [ ]:
Austin_Trips_Df=Austin_Trips_Df.withColumn('Speed',(Austin_Trips_Df['Trip Distance']/Austin_Trips_Df['Trip Duration'])*3.6)

In [ ]:
Austin_Trips_Df.describe(['Speed']).show()